In [8]:
using Plots
using LaTeXStrings
using Printf
using BoundaryValueDiffEq

In [9]:
domain = (0.0, 1.0)
α = 1.0
β = 2.0
ϵ = 0.1
ϵ2 = 0.01;

In [10]:
# Differential equation definition
function prob2!(dv, v, p, x)
    @views dv[1] = v[2]
    @views dv[2] = (v[2] + 2*x) / p  # Solve for the current epsilon
    return dv
end

prob2! (generic function with 1 method)

In [11]:
# Boundary conditions
function bc2!(residual, u, p, x)
    residual[1] = u[1][1] - α
    residual[2] = u[end][1] - β
end

bc2! (generic function with 1 method)

In [12]:
# Define the problem and solve for the current epsilon
u0 = [α, 0.0]

bvp1 = BVProblem(prob2!, bc2!, u0, domain, ϵ)
bvp2 = BVProblem(prob2!, bc2!, u0, domain, ϵ2)

sol1 = solve(bvp1, MIRK4(), dt = 0.01);
sol2 = solve(bvp2, MIRK4(), dt = 0.01);

In [13]:
x = collect(LinRange(0,1,1000))
z(x) =(x-1) / ϵ

A = β - α
B = 2

composite(x, ϵ) = α + β - x^2 + A*(exp.(z(x)) - 1) - 2*x + ϵ*(B*(exp.(z(x))-1) -2*x);

In [15]:
theme(:dao::Symbol)
plt = plot(sol1,
    idxs=[1],
    label="exact "*L"$(\varepsilon=%$ϵ)$",
    xlabel = L"$x$",  # Explicitly set the x-axis label
    ylabel = L"$y(x)$",
    title = "Problem 2 Exact Solutions",
    legend = :best,
    titlefontsize = 20,
    tickfontsize = 12,
    legendfontsize = 10,
    yguidefontsize = 15,
    xguidefontsize = 15,
    line = (3, :solid),
    right_margin = 2 * Plots.mm)
plot!(plt, sol2,
    idxs=[1],
    label="exact "*L"$(\varepsilon=%$ϵ2)$",
    xlabel = L"$x$",
    line = (3, :solid))
savefig(plt, "Aero_HW6_Prob2_exact.pdf")

"/home/belster12/JuliaFiles/Aero/hw6/Aero_HW6_Prob2_exact.pdf"